In [30]:
# imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import requests
import pickle
import io

In [31]:
# path to the dataset (replace with your own path) MAKE SURE YOU CHANGE THE SLASHES
path = "C:/Users/Asa/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1"

In [32]:
import preprocessing as prep
target_size = (150, 150)
grayscale = True
X, y = prep.get_X_y(path=path, percent=0.5, target_size=target_size, grayscale=grayscale)

In [33]:
no_output_classes = len(y.columns)

In [34]:
y[:5]

,battery,cardboard,clothes,glass,metal,paper,plastic,trash
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
X.shape

(2608, 150, 150)

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [37]:
print("Train Lengths:", len(X_train), len(y_train))
print("Test Lengths:", len(X_test), len(y_test))

Train Lengths: 1956 1956
Test Lengths: 652 652


In [39]:
# defining the model
from tensorflow.keras import layers

input_shape = target_size
if grayscale:
    input_shape = input_shape + (1,)
else:
    input_shape = input_shape + (3,)

# removing images from the dataset
# dropout: randomly remove nodes in each layer to create gaps
# normalization layer
# augmentation
# convert to grayscale <- 
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(no_output_classes, activation='sigmoid')
])

In [40]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [41]:
batch_size = 32
epochs = 10
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)

Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 26s 370ms/step - accuracy: 0.2572 - loss: 2.2120 - val_accuracy: 0.1028 - val_loss: 2.0949
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 28s 445ms/step - accuracy: 0.4347 - loss: 1.6170 - val_accuracy: 0.1074 - val_loss: 2.0505
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 26s 408ms/step - accuracy: 0.5035 - loss: 1.4402 - val_accuracy: 0.1794 - val_loss: 2.0557
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 29s 462ms/step - accuracy: 0.5332 - loss: 1.3300 - val_accuracy: 0.1963 - val_loss: 1.9927
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 34s 346ms/step - accuracy: 0.5501 - loss: 1.2218 - val_accuracy: 0.2147 - val_loss: 1.9422
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 22s 348ms/step - accuracy: 0.6333 - loss: 1.0647 - val_accuracy: 0.3221 - val_loss: 1.8137
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 21s 342ms/step - accuracy: 0.6664 - loss: 0.9506 - val_accuracy: 0.1887 - val_loss: 2.5158
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 21s 343ms/step - accuracy: 0.5944 - loss: 1.1349 - val_accu

In [42]:
# Evaluate the model using the testing data
model.evaluate(X_test, y_test)

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.4561 - loss: 1.5874


[1.6147503852844238, 0.44938650727272034]

In [43]:
# Saving the model
with open("initial_model.pkl", "wb") as f:
    pickle.dump(model, f)

### Testing the model

In [44]:
# reloaing the model
import pickle
with open("initial_model.pkl", "rb") as f:
    model = pickle.load(f)

In [45]:
# preprocess a new image 
import preprocessing as prep
image_path ="clothes_test_image.jpg"
X_input = prep.get_X([image_path], target_size=(150, 150))

In [46]:
import pandas as pd
output = model.predict(X_input)
pd.DataFrame(output, columns=prep.encoded_y_cols)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step


c:\Users\Asa\miniconda3\envs\dev\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer_3
Received: inputs=('Tensor(shape=(1, 150, 150))', ())
  warnings.warn(msg)


,battery,cardboard,clothes,glass,metal,paper,plastic,trash
0,0.96198,0.461258,0.997945,0.684569,0.696039,0.385976,0.654634,0.036925


In [47]:
import numpy as np
predicted_class = np.argmax(output[0])
prep.encoded_y_cols[predicted_class]

'clothes'

In [48]:
prep.get_prediction(image_path=image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


c:\Users\Asa\miniconda3\envs\dev\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer_3
Received: inputs=('Tensor(shape=(1, 150, 150))', ())
  warnings.warn(msg)


'clothes'